### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the openml resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. 
```

In [1]:
from sklearn.datasets import fetch_openml

In [2]:
wages = fetch_openml(data_id=534, as_frame=True)

In [7]:
df = wages.frame

In [8]:
df

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8.0,no,female,21.0,not_member,5.10,35.0,Hispanic,Other,Manufacturing,Married
1,9.0,no,female,42.0,not_member,4.95,57.0,White,Other,Manufacturing,Married
2,12.0,no,male,1.0,not_member,6.67,19.0,White,Other,Manufacturing,Unmarried
3,12.0,no,male,4.0,not_member,4.00,22.0,White,Other,Other,Unmarried
4,12.0,no,male,17.0,not_member,7.50,35.0,White,Other,Other,Married
...,...,...,...,...,...,...,...,...,...,...,...
529,18.0,no,male,5.0,not_member,11.36,29.0,White,Professional,Other,Unmarried
530,12.0,no,female,33.0,not_member,6.10,51.0,Other,Professional,Other,Married
531,17.0,no,female,25.0,member,23.25,48.0,Other,Professional,Other,Married
532,12.0,yes,male,13.0,member,19.88,31.0,White,Professional,Other,Married


In [16]:
df['MARR'].value_counts()

Married      350
Unmarried    184
Name: MARR, dtype: int64

#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np
import plotly.express as px
import pandas as pd
import warnings

In [18]:
cat_cols = ['SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']

In [19]:
ndf = pd.get_dummies(df, columns = cat_cols)

In [21]:
ndf.head()

,EDUCATION,EXPERIENCE,WAGE,AGE,SOUTH_no,SOUTH_yes,SEX_female,SEX_male,UNION_member,UNION_not_member,...,OCCUPATION_Management,OCCUPATION_Other,OCCUPATION_Professional,OCCUPATION_Sales,OCCUPATION_Service,SECTOR_Construction,SECTOR_Manufacturing,SECTOR_Other,MARR_Married,MARR_Unmarried
0,8.0,21.0,5.10,35.0,1,0,1,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,9.0,42.0,4.95,57.0,1,0,1,0,0,1,...,0,1,0,0,0,0,1,0,1,0
2,12.0,1.0,6.67,19.0,1,0,0,1,0,1,...,0,1,0,0,0,0,1,0,0,1
3,12.0,4.0,4.00,22.0,1,0,0,1,0,1,...,0,1,0,0,0,0,0,1,0,1
4,12.0,17.0,7.50,35.0,1,0,0,1,0,1,...,0,1,0,0,0,0,0,1,1,0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(ndf.drop('WAGE', axis = 1), np.log1p(ndf.WAGE), random_state=42, test_size = 0.3)

In [30]:
model_1 = Ridge().fit(X_train, y_train)
model_1_coefs = model_1.coef_
print(f'Ridge Coefs: {np.round(model_1_coefs, 2)}')

Ridge Coefs: [ 0.09  0.05 -0.04  0.03 -0.03 -0.09  0.09  0.11 -0.11 -0.03 -0.03  0.06
 -0.01  0.19 -0.06  0.15 -0.1  -0.16  0.03  0.04 -0.08  0.02 -0.02]


In [31]:
alphas = [0.001, 1.0, 10.0, 100.0]

In [37]:
coef_list = []

for alpha in alphas:
    ridge = Ridge(alpha = alpha)
    ridge.fit(X_train, y_train)
    coef_list.append(list(ridge.coef_))
    
print('For alpha = 100.0 we have the following coefficients:')
list(zip(X_train.columns, coef_list[-1]))

For alpha = 100.0 we have the following coefficients:


[('EDUCATION', 0.04597853432520751),
 ('EXPERIENCE', -0.011162009917497976),
 ('AGE', 0.021812628817323425),
 ('SOUTH_no', 0.02553164237067024),
 ('SOUTH_yes', -0.025531642370670207),
 ('SEX_female', -0.059866996251041985),
 ('SEX_male', 0.059866996251041964),
 ('UNION_member', 0.05477943944182321),
 ('UNION_not_member', -0.05477943944182328),
 ('RACE_Hispanic', -0.008560845640496476),
 ('RACE_Other', -0.018430802618836558),
 ('RACE_White', 0.026991648259333045),
 ('OCCUPATION_Clerical', -0.02052252844934937),
 ('OCCUPATION_Management', 0.04458662024655539),
 ('OCCUPATION_Other', 0.00967718546124391),
 ('OCCUPATION_Professional', 0.04339430223592978),
 ('OCCUPATION_Sales', -0.024600748828942663),
 ('OCCUPATION_Service', -0.052534830665436606),
 ('SECTOR_Construction', 0.012260013130771422),
 ('SECTOR_Manufacturing', 0.023914429994528048),
 ('SECTOR_Other', -0.03617444312529954),
 ('MARR_Married', 0.016425197519411492),
 ('MARR_Unmarried', -0.016425197519411495)]

In [39]:
X = ndf.drop('WAGE', axis = 1)
y = ndf['WAGE']

In [42]:
########ridge with small regularization penalty

ridge_low_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 5, include_bias= False)), 
                     ('ridge_low', Ridge(alpha = 0.1))])
ridge_low_pipe.fit(X, y)
ridge_low_preds = ridge_low_pipe.predict(X)

C:\Users\oyeye\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


In [43]:
ridge_low_preds

array([ 6.59184010e+00, -8.52686635e+00,  5.03851414e+00, -4.71813582e+00,
        1.64339165e+01, -5.57654188e+00,  2.58765121e+00,  4.96887425e+00,
        1.05473509e+01,  4.96887425e+00,  1.35029095e+01,  1.51602676e+01,
        1.02070752e+01, -3.62550995e+00,  1.42788347e+01,  1.08479019e+01,
        1.06981550e+01,  4.20316147e+00,  1.98743821e+01,  1.34340300e+01,
        1.54613811e+01,  5.06842407e+00,  4.96887425e+00,  2.14657615e+01,
        1.19507129e+01,  1.04527419e+01,  2.63819291e+00, -5.59677417e+00,
        5.20451862e+00,  4.02048361e+00,  1.55594942e+01,  2.98647357e+00,
        1.22165835e+01,  6.52038171e+00,  1.24274081e+01,  9.71885695e+00,
        3.08648313e+01,  2.82472304e+00, -5.17146711e+00,  1.64184293e+01,
       -3.74420962e-01,  1.99089852e+01,  6.80890952e+00, -1.78166048e+01,
        2.27029526e+00, -1.79097591e+01,  6.03601816e+00,  1.82985961e+01,
        1.38282926e+01,  2.92109858e+00,  2.07097153e+00,  4.38332024e+00,
        6.92592944e+00, -

In [44]:
########ridge with high regularization penalty

ridge_high_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 5, include_bias= False)), 
                     ('ridge_high', Ridge(alpha = 1000))])
ridge_high_pipe.fit(X, y)
ridge_high_preds = ridge_high_pipe.predict(X)

C:\Users\oyeye\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.27381e-19): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True,


In [45]:
ridge_high_preds

array([ 5.14633463,  6.29400853,  7.18288668,  6.46876379,  8.38864795,
       12.85667083,  4.41552239,  8.61822869, 14.61653553,  8.61822869,
       10.95395284, 11.57531147,  7.13190118,  6.16412825, 19.12687955,
        9.26767455,  8.97228212, 22.19757707,  4.98198587, 19.17027692,
        4.66024969,  7.78188687,  8.61822869,  4.43796206,  7.85377966,
        8.86751851,  8.38747701,  6.44109635,  2.3886863 ,  6.75332349,
       11.54965799,  6.19973622,  4.32607031,  5.58372454,  8.82313628,
       10.86685294,  6.80843617,  8.82984521,  5.60142243, 11.36009148,
        3.04259727, 10.60019478,  8.6065711 ,  4.42219072,  4.01221121,
        4.71657364, 13.34454227,  5.49130399,  9.10468814,  7.35132825,
        7.40329557, 10.73731162,  4.36967758,  4.94119376,  3.97928906,
        5.21366459,  8.23997063,  6.97039765, 10.0899348 ,  5.57651375,
        4.98880492, 10.75936221, 11.36683392, 12.75795406,  4.41329493,
        8.96319166, 13.47530837, 11.61954142,  9.33395791,  8.97

In [48]:
scaled_pipe = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge())]).fit(X_train, y_train)
train_preds = scaled_pipe.predict(X_train)
test_preds = scaled_pipe.predict(X_test)
train_mse = mean_squared_error(y_train, train_preds)
test_mse = mean_squared_error(y_test, test_preds)

In [49]:
train_mse

0.1352300474504382

In [50]:
test_mse

0.14463248680103416

In [51]:
pipe = Pipeline([('scale', StandardScaler()), ('ridge', Ridge())])

In [52]:
param_dict = {'ridge__alpha': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}

In [53]:
grid_2 = GridSearchCV(pipe, param_grid=param_dict)
grid_2.fit(X_train, y_train)
train_preds = grid_2.predict(X_train)
test_preds = grid_2.predict(X_test)
model_2_train_mse = mean_squared_error(y_train, train_preds)
model_2_test_mse = mean_squared_error(y_test, test_preds)
model_2_best_alpha = grid_2.best_params_

In [55]:
model_2_train_mse

0.1365495971621039

In [54]:
model_2_test_mse

0.14840849768895062

In [56]:
model_2_best_alpha

{'ridge__alpha': 100.0}